In [ ]:
!pip install tensorflow

In [ ]:
!pip install PIL

In [ ]:
!pip install scipy

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:

# Parameter
IMG_SIZE = (224, 224)  # Ukuran gambar untuk MobileNetV2
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.001 

# Direktori dataset
train_dir = "train"
test_dir = "test"

# Augmentasi Data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Generator Data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

# Informasi jumlah kelas
num_classes = len(train_generator.class_indices)

Found 6881 images belonging to 116 classes.
Found 1292 images belonging to 116 classes.


In [ ]:


# Model Transfer Learning
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Bekukan layer pada model dasar untuk transfer learning

# Tambahkan lapisan khusus
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Dropout untuk mencegah overfitting
x = Dense(256, activation="relu")(x)
predictions = Dense(num_classes, activation="softmax")(x)

# Buat model
model = Model(inputs=base_model.input, outputs=predictions)

# Kompilasi model untuk transfer learning
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Callback
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [5]:

# Pelatihan awal (transfer learning)
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator,
    callbacks=[reduce_lr, early_stopping]
)

C:\Users\admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1865 - loss: 3.6482

C:\Users\admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


216/216 ━━━━━━━━━━━━━━━━━━━━ 339s 2s/step - accuracy: 0.1871 - loss: 3.6445 - val_accuracy: 0.5573 - val_loss: 1.6358 - learning_rate: 0.0010
Epoch 2/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 300s 1s/step - accuracy: 0.5081 - loss: 1.7170 - val_accuracy: 0.6602 - val_loss: 1.1842 - learning_rate: 0.0010
Epoch 3/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 305s 1s/step - accuracy: 0.5987 - loss: 1.3232 - val_accuracy: 0.7098 - val_loss: 0.9994 - learning_rate: 0.0010
Epoch 4/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 301s 1s/step - accuracy: 0.6337 - loss: 1.2143 - val_accuracy: 0.7167 - val_loss: 0.9735 - learning_rate: 0.0010
Epoch 5/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 248s 1s/step - accuracy: 0.6748 - loss: 1.0802 - val_accuracy: 0.7314 - val_loss: 0.8927 - learning_rate: 0.0010
Epoch 6/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 227s 1s/step - accuracy: 0.6887 - loss: 1.0014 - val_accuracy: 0.7384 - val_loss: 0.8699 - learning_rate: 0.0010
Epoch 7/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 221s 1s/step - accuracy: 0.7019 - loss: 0.9577 - val_

In [6]:


# Fine-tuning
# Buka beberapa lapisan terakhir dari base model
base_model.trainable = True
for layer in base_model.layers[:-20]:  # Bekukan semua kecuali 20 lapisan terakhir
    layer.trainable = False

# Kompilasi ulang model dengan learning rate yang lebih kecil
model.compile(optimizer=Adam(learning_rate=FINE_TUNE_LR),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Fine-tuning model
history_fine_tune = model.fit(
    train_generator,
    epochs=FINE_TUNE_EPOCHS,
    validation_data=test_generator,
    callbacks=[reduce_lr, early_stopping]
)

# Evaluasi model
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Epoch 1/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 253s 1s/step - accuracy: 0.5647 - loss: 1.5646 - val_accuracy: 0.7755 - val_loss: 0.7222 - learning_rate: 1.0000e-05
Epoch 2/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 251s 1s/step - accuracy: 0.6602 - loss: 1.1300 - val_accuracy: 0.7802 - val_loss: 0.7063 - learning_rate: 1.0000e-05
Epoch 3/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 249s 1s/step - accuracy: 0.6971 - loss: 0.9919 - val_accuracy: 0.7910 - val_loss: 0.6951 - learning_rate: 1.0000e-05
Epoch 4/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 245s 1s/step - accuracy: 0.7172 - loss: 0.9016 - val_accuracy: 0.7933 - val_loss: 0.6848 - learning_rate: 1.0000e-05
Epoch 5/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 221s 1s/step - accuracy: 0.7234 - loss: 0.8897 - val_accuracy: 0.7988 - val_loss: 0.6673 - learning_rate: 1.0000e-05
Epoch 6/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 215s 997ms/step - accuracy: 0.7388 - loss: 0.8255 - val_accuracy: 0.7972 - val_loss: 0.6580 - learning_rate: 1.0000e-05
Epoch 7/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 215s 993ms/step

In [7]:
model.save("model_10_20_epochs.h5")  # Menyimpan model ke file my_model.h5